<a href="https://colab.research.google.com/github/robicastilho/remedio_certo/blob/main/remedio_certo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Instale a SDK Google Gen AI
!pip -q install google-genai

#Imports e configuração do Colab
from google import genai
from google.genai import types
from google.colab import files
from google.colab import userdata
import os

#Configure o acesso ao Vertex AI (Gemini Flash 2.0)
#os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"
#os.environ["GOOGLE_CLOUD_PROJECT"] = "SEU_PROJECT_ID"
#os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
#model_name = "gemini-2.0-flash"
model_name = "gemini-2.5-flash-preview-04-17"

#Autentique-se no Google Cloud
# Execute esta célula e siga as instruções no navegador
#!gcloud auth application-default login

#Crie o cliente
client = genai.Client()  # :contentReference[oaicite:1]{index=1}

#Widget de upload
print("👆 Escolha o arquivo de imagem (receita médica ou não):")
uploaded = files.upload()
image_path = next(iter(uploaded.keys()))

#Envie a imagem à API de arquivos do Gen AI
file = client.files.upload(file=image_path)  # :contentReference[oaicite:2]{index=2}

#cria variavel para armazenar o prompt
especialidade_do_prescritor = input('Especialidade: ')
sintomas_paciente = input('Sintomas: ')

prompt = f"""
Você é um Farmacêutico altamente experiente e especializado em decifrar receitas médicas manuscritas, com profundo conhecimento em farmacologia, caligrafia médica e abreviações comuns.

**Você já tem a informação de que a especialidade médica do prescritor é {especialidade_do_prescritor} e os sintomas do paciente {sintomas_paciente}.**

Sua tarefa principal é analisar uma imagem de uma receita médica manuscrita e extrair as informações cruciais sobre os medicamentos prescritos.

**Para cada item identificado na receita, forneça as seguintes informações em um formato claro e objetivo:**

* **Medicamento:** Nome do medicamento (se houver incerteza, inclua possíveis alternativas e uma observação).
* **Posologia:** Dose, via de administração e frequência (ex: "500mg, via oral, a cada 8 horas").
* **Duração do Tratamento:** Período pelo qual o medicamento deve ser administrado (ex: "por 7 dias").
* **Grau de Confiança:** (ALTO, MÉDIO, BAIXO) na sua identificação, para cada campo (Medicamento, Posologia, Duração).
* **Observações/Incertezas:** Detalhes sobre qualquer dificuldade na leitura, caracteres ilegíveis ou suposições feitas.

---

**Passos para a identificação (em ordem de prioridade):**

1.  **Decifração Direta:** Comece lendo e transcrevendo o que o médico escreveu com a maior precisão possível.
2.  **Análise de Contexto e Caligrafia:** Utilize seu conhecimento em farmacologia, abreviações comuns em receitas e, crucialmente, a **informação da especialidade médica e sintomas do paciente** para inferir palavras ou partes ilegíveis.
3.  **Busca por Similares (Grafia):** Se a palavra for incerta ou o medicamento encontrado não seja para tratamento dos sintomas informados, pesquise por nomes de medicamentos ou termos de posologia com grafia similar ou fonética próxima que se encaixem no contexto.
4.  **Identificação de Ilegíveis:** Se, após os passos anteriores, uma parte da receita permanecer impossível de decifrar, classifique-a como "Não identificado" ou "Ilegível" e atribua um grau de confiança "BAIXO". **Nunca invente informações ou nomes de medicamentos.**
5.  **Validação e Refinamento Contextual:**
    * Com base no nome do medicamento que você identificou (mesmo que com confiança baixa), **pesquise no (google_search) por medicamentos que tenham nomes iguais ou muito similares**.
    * Simultaneamente, avalie se os medicamentos encontrados nessa pesquisa e o medicamento originalmente identificado **possuem ação terapêutica compatível com os sintomas informados pelo paciente e são comumente prescritos por médicos com a mesma especialidade** para essa condição.
    * Se esta análise contextual sugerir fortemente um medicamento diferente do inicialmente decifrado (mas graficamente similar e contextualmente mais plausível), **corrija a informação do medicamento** e ajuste o Grau de Confiança para cima, justificando a alteração em "Observações/Incertezas". Caso a compatibilidade seja baixa, mas a legibilidade da receita persista, mantenha a incerteza e o Grau de Confiança original ou até o diminua.

---
"""
#print(prompt)

#Chame o modelo multimodal
response = client.models.generate_content(
    model=model_name,
    contents=[
        prompt,
        file
    ],
)

#Exiba a resposta
print("Resposta do Gemini Flash 2.0:")
print(response.text)

print("""\n\n**Aviso Importante e Limitação de Responsabilidade:**

**O resultado desta análise é apenas informativo e visa auxiliar na decifração da caligrafia médica. Sob hipótese alguma esta transcrição representa uma sugestão, recomendação, orientação ou substituição de aconselhamento médico ou farmacêutico profissional. Você não deve administrar medicamentos com base nesta análise. Consulte sempre um profissional de saúde qualificado (médico ou farmacêutico) para confirmar e validar qualquer informação relacionada à sua saúde e medicação.**

**Sua prioridade é a precisão e a segurança do paciente, mesmo que isso signifique relatar incerteza. Seja objetivo e profissional em todas as suas respostas.**""")


👆 Escolha o arquivo de imagem (receita médica ou não):


Saving receita-medica-exemplo-coceira-genital-urologista.jpg to receita-medica-exemplo-coceira-genital-urologista (3).jpg
Especialidade: urologista
Sintomas: coceira genital
Resposta do Gemini Flash 2.0:
Com base na análise da imagem da receita manuscrita e considerando o contexto fornecido (especialidade urologista, sintoma coceira genital), apresento a decifração:

**Item 1:**

*   **Medicamento:** Loprox (Ciclopirox Olamina)
*   **Posologia:** 2 vezes ao dia
*   **Duração do Tratamento:** Por 7 dias
*   **Grau de Confiança:**
    *   Medicamento: MÉDIO
    *   Posologia: ALTO
    *   Duração do Tratamento: ALTO
*   **Observações/Incertezas:** O nome do medicamento escrito na receita se assemelha muito a "Lopox". No entanto, "Loprox" (nome comercial para Ciclopirox Olamina) é um antifúngico tópico de uso comum para tratar infecções fúngicas que causam coceira genital (como tinea cruris - "micose na virilha") e é frequentemente prescrito por urologistas. Graficamente, a diferença entr